# LDA - AdaBoost Pipeline 

## Import necessary packages

In [14]:
import time
import optuna
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier

from dataset.dataset import Dataset
from constants import CLEANED_DATASET_PATH

## Constants

In [15]:
RANDOM_SEED = 0

## Load the Hate Speech Filipino dataset from Hugging Face

In [16]:
dataset = Dataset()
X, Y = dataset.load_from_file(CLEANED_DATASET_PATH)

Data loaded from dataset/cleaned_dataset.pkl


In [17]:
# Re-split the dataset into training, validation, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=RANDOM_SEED)

## Vectorize the texts to be able to perform LDA

In [18]:
# Define the CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2)
X_train_vector = vectorizer.fit_transform(X_train)
X_val_counts = vectorizer.transform(X_val)
X_test_counts = vectorizer.transform(X_test)

## Perform Latent Dirichlet Allocation on the training set

In [19]:
N_TOPICS = 10
print(f"Performing Latent Dirichlet Allocation for {N_TOPICS} topics")
lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = lda.fit_transform(X_train_vector)
X_val_topics = lda.transform(X_val_counts)
X_test_topics = lda.transform(X_test_counts)
print(f"Done performing Latent Dirichlet Allocation for {N_TOPICS} topics")

Performing Latent Dirichlet Allocation for 10 topics
Done performing Latent Dirichlet Allocation for 10 topics


## Train an AdaBoost model then evaluate its performance

In [20]:
# Define the objective function for Optuna
def create_objective(X_train, Y_train, X_test, Y_test):
    def objective(trial):
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)
        learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2)
        ada = AdaBoostClassifier(n_estimators=n_estimators, 
                                 learning_rate=learning_rate, 
                                 algorithm="SAMME", 
                                 random_state=RANDOM_SEED)
        ada.fit(X_train, Y_train)
        Y_pred = ada.predict(X_test)
        score = accuracy_score(Y_test, Y_pred)
        return score
    return objective

In [21]:
# Create a study object
study = optuna.create_study(direction="maximize", study_name=f"LDA_AdaBoost_Pipeline")

# Create the study objective
objective = create_objective(X_train_topics, Y_train, X_val_topics, Y_val)

# Execute an optimization
study.optimize(objective, n_trials=20, n_jobs=-1)

# Print the best trial results
print(f"Best Accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2024-04-30 21:58:07,010] A new study created in memory with name: LDA_AdaBoost_Pipeline
[I 2024-04-30 21:58:10,742] Trial 7 finished with value: 0.5482123510292525 and parameters: {'n_estimators': 150, 'learning_rate': 0.00020552208091210978}. Best is trial 7 with value: 0.5482123510292525.
[I 2024-04-30 21:58:13,354] Trial 6 finished with value: 0.5525460455037919 and parameters: {'n_estimators': 262, 'learning_rate': 0.009085275162654003}. Best is trial 6 with value: 0.5525460455037919.
[I 2024-04-30 21:58:13,851] Trial 3 finished with value: 0.5482123510292525 and parameters: {'n_estimators': 282, 'learning_rate': 0.004341526648646872}. Best is trial 6 with value: 0.5525460455037919.
[I 2024-04-30 21:58:14,207] Trial 4 finished with value: 0.5482123510292525 and parameters: {'n_estimators': 295, 'learning_rate': 0.0016945934749096963}. Best is trial 6 with value: 0.5525460455037919.
[I 2024-04-30 21:58:16,854] Trial 5 finished with value: 0.5482123510292525 and parameters: {'n_es

Best Accuracy: 0.638136511375948
Best hyperparameters: {'n_estimators': 853, 'learning_rate': 0.009446661480410042}


In [22]:
best_n_estimators = study.best_trial.params["n_estimators"]
best_learning_rate = study.best_trial.params["learning_rate"]

ada = AdaBoostClassifier(n_estimators=best_n_estimators, 
                         learning_rate=best_learning_rate,
                         algorithm="SAMME",
                         random_state=RANDOM_SEED)
ada.fit(X_train_topics, Y_train)
Y_pred = ada.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
print(f"Best LDA-RF Model Accuracy: {score}")

Best LDA-RF Model Accuracy: 0.6599891716296697


In [23]:
from sklearn.decomposition import NMF

N_TOPICS = 20
print(f"Performing Non-negative Matrix Factorization for {N_TOPICS} topics")
nmf = NMF(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = nmf.fit_transform(X_train_vector)
X_val_topics = nmf.transform(X_val_counts)
X_test_topics = nmf.transform(X_test_counts)
print(f"Done performing Non-negative Matrix Factorization for {N_TOPICS} topics")

Performing Non-negative Matrix Factorization for 20 topics
Done performing Non-negative Matrix Factorization for 20 topics


In [25]:
ada = AdaBoostClassifier(n_estimators=best_n_estimators, 
                         learning_rate=best_learning_rate,
                         algorithm="SAMME",
                         random_state=RANDOM_SEED)
ada.fit(X_train_topics, Y_train)
Y_pred = ada.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
print(f"Best NMF-RF Model Accuracy: {score}")

Best NMF-RF Model Accuracy: 0.6513264753654575
